# 🧮 5. Data Preprocessing
## 5.1 Tujuan

Tahap Data Preprocessing bertujuan untuk mengubah data audio mentah menjadi data numerik yang siap digunakan untuk proses modeling. Proses ini meliputi pembacaan file audio, normalisasi sinyal, penghapusan noise, pemotongan bagian diam, serta ekstraksi fitur statistik time series seperti mean, standard deviation, RMS, dan zero crossing rate (ZCR).

## 5.2 Langkah-Langkah Preprocessing
### a. Import Library yang Dibutuhkan

In [1]:
import os
import numpy as np
import librosa
import soundfile as sf       
import matplotlib.pyplot as plt
from tqdm import tqdm
from pathlib import Path
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
import pickle

## b.Lakukan preprocessing

membuat fungsi preprocessing audio, yang akan:

✅ Membaca semua file .wav dari masing-masing orang dan kelas

✅ Normalisasi amplitudo

✅ Trimming bagian diam

✅ Zero-padding agar durasi seragam (misalnya 2 detik @16kHz = 32000 sampel)

✅ Simpan hasil bersih ke folder preprocces_person1,2

In [2]:
import os
import librosa
import numpy as np
from tqdm import tqdm

# ==========================
# PATH DATASET
# ==========================
DATASET_PERSON1 = r"D:\KULIAH\SEMESTER 5\Program Saint Data\Uranus\myfirstbook\Audio_recognition\Dataset_Voice_pertama"
DATASET_PERSON2 = r"D:\KULIAH\SEMESTER 5\Program Saint Data\Uranus\myfirstbook\Audio_recognition\Dataset_Voice_kedua"

# ==========================
# PARAMETER PREPROCESSING
# ==========================
TARGET_SR = 16000          
TARGET_DURATION = 3.0      
TARGET_LEN = int(TARGET_SR * TARGET_DURATION)

# ==========================
# FUNGSI PREPROCESSING
# ==========================
def preprocess_audio(file_path):
    try:
        y, sr = librosa.load(file_path, sr=TARGET_SR)
        y = librosa.util.normalize(y)
        y, _ = librosa.effects.trim(y, top_db=25)

        # Balancing durasi
        if len(y) < TARGET_LEN:
            y = np.pad(y, (0, TARGET_LEN - len(y)), mode='constant')
        else:
            y = y[:TARGET_LEN]

        return y
    except Exception as e:
        print(f"❌ Error memproses {file_path}: {e}")
        return None

# ==========================
# PREPROCESS DATASET PERSON 1
# ==========================
print("\n🎤 Memproses: Dataset_Voice_pertama")
data_person1 = {}  # dictionary: {'buka': [array, ...], 'tutup': [array, ...]}

valid_folders = [f for f in os.listdir(DATASET_PERSON1) if f.endswith("_wav")]

for label_folder in valid_folders:
    label_path = os.path.join(DATASET_PERSON1, label_folder)
    if not os.path.isdir(label_path):
        continue

    label_clean = label_folder.replace("_wav", "")
    data_person1[label_clean] = []

    for file in tqdm(os.listdir(label_path), desc=f"Person1/{label_clean}"):
        if file.endswith(".wav"):
            in_path = os.path.join(label_path, file)
            y = preprocess_audio(in_path)
            if y is not None:
                data_person1[label_clean].append(y)

# ==========================
# PREPROCESS DATASET PERSON 2
# ==========================
print("\n🎙️ Memproses: Dataset_Voice_kedua")
data_person2 = {}  # dictionary: {'buka': [array, ...], 'tutup': [array, ...]}

for label_folder in os.listdir(DATASET_PERSON2):
    label_path = os.path.join(DATASET_PERSON2, label_folder)
    if not os.path.isdir(label_path):
        continue

    data_person2[label_folder] = []

    for file in tqdm(os.listdir(label_path), desc=f"Person2/{label_folder}"):
        if file.endswith(".wav"):
            in_path = os.path.join(label_path, file)
            y = preprocess_audio(in_path)
            if y is not None:
                data_person2[label_folder].append(y)

print("\n✅ Preprocessing selesai.")



🎤 Memproses: Dataset_Voice_pertama


Person1/Buka:   0%|                              | 0/50 [00:00<?, ?it/s]

Person1/Buka:   2%|▍                     | 1/50 [00:01<01:37,  2.00s/it]

Person1/Buka:  60%|████████████▌        | 30/50 [00:02<00:01, 19.78it/s]

Person1/Buka: 100%|█████████████████████| 50/50 [00:02<00:00, 22.87it/s]

...

Person2/Buka: 100%|████████████████████| 49/49 [00:00<00:00, 197.86it/s]

Person2/Tutup:   0%|                             | 0/48 [00:00<?, ?it/s]

Person2/Tutup:  54%|██████████▎        | 26/48 [00:00<00:00, 249.23it/s]

Person2/Tutup: 100%|███████████████████| 48/48 [00:00<00:00, 250.24it/s]


✅ Preprocessing selesai.


### c. Fungsi Ekstraksi Fitur Statistik

Fungsi ini akan membaca setiap file .wav, melakukan preprocessing dasar, lalu menghitung fitur statistik dari sinyal time domain.

In [3]:
import os
import numpy as np
import pandas as pd
import librosa

SAMPLE_RATE = 16000

def extract_features(y, sr=SAMPLE_RATE):
    features = []

    # 1. Statistik dasar
    features += [
        np.mean(y),
        np.std(y),
        np.var(y),
        np.mean((y - np.mean(y))**3)/(np.std(y)**3 + 1e-6),
        np.mean((y - np.mean(y))**4)/(np.std(y)**4 + 1e-6),
        np.sqrt(np.mean(y**2)),
        np.mean(librosa.feature.zero_crossing_rate(y)),
        np.std(librosa.feature.zero_crossing_rate(y)),
        np.max(y) - np.min(y)
    ]

    # 2. Spektral
    spec_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)
    spec_bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    spec_contrast = librosa.feature.spectral_contrast(y=y, sr=sr)
    spec_rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    spec_flatness = librosa.feature.spectral_flatness(y=y)
    chroma = librosa.feature.chroma_stft(y=y, sr=sr)

    features += [
        np.mean(spec_centroid), np.std(spec_centroid),
        np.mean(spec_bandwidth), np.std(spec_bandwidth),
        np.mean(spec_contrast), np.std(spec_contrast),
        np.mean(spec_rolloff), np.std(spec_rolloff),
        np.mean(spec_flatness), np.std(spec_flatness),
        np.mean(chroma), np.std(chroma)
    ]

    # 3. MFCC (40 koef)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)

    for i in range(40):
        features.append(np.mean(mfcc[i]))
    for i in range(40):
        features.append(np.std(mfcc[i]))

    # 4. Delta MFCC (40)
    mfcc_delta = librosa.feature.delta(mfcc)

    for i in range(40):
        features.append(np.mean(mfcc_delta[i]))
    for i in range(40):
        features.append(np.std(mfcc_delta[i]))

    # 5. Temporal / Energy
    rms = librosa.feature.rms(y=y)
    features.append(np.mean(rms))
    features.append(np.std(rms))

    onset_env = librosa.onset.onset_strength(y=y, sr=sr)
    features.append(np.mean(onset_env))
    features.append(np.std(onset_env))

    tempo = librosa.beat.tempo(onset_envelope=onset_env, sr=sr)
    features.append(float(tempo[0]))

    autocorr = np.correlate(y, y, mode='full')
    mid = len(autocorr)//2
    features.append(np.argmax(autocorr[mid+1:]) + 1)

    return features


### d. Looping untuk Mengambil Fitur dari Semua File

Kita akan mengambil semua file dari folder buka dan tutup, lalu menambahkan label untuk tiap file.

In [4]:
import pandas as pd

data_rows = []
labels = []

all_data = [
    ("naufal", data_person1),
    ("harits", data_person2)
]

for person_tag, data_dict in all_data:
    for label, audio_list in data_dict.items():
        for y_audio in audio_list:
            feats = extract_features(y_audio)
            data_rows.append(feats)
            labels.append(f"{person_tag}_{label}") 


# ==========================
# 1. Kolom Statistik dasar
# ==========================
columns = [
    "mean","std","var","skew","kurtosis","rms_global",
    "zcr_mean","zcr_std","amplitude_range"
]

# ==========================
# 2. Spektral
# ==========================
columns += [
    "spec_centroid_mean","spec_centroid_std",
    "spec_bandwidth_mean","spec_bandwidth_std",
    "spec_contrast_mean","spec_contrast_std",
    "spec_rolloff_mean","spec_rolloff_std",
    "spec_flatness_mean","spec_flatness_std",
    "chroma_mean","chroma_std"
]

# ==========================
# 3. MFCC 40 (mean + std)
# ==========================
for i in range(1, 41):
    columns.append(f"mfcc_{i}_mean")

for i in range(1, 41):
    columns.append(f"mfcc_{i}_std")

# ==========================
# 4. Delta MFCC 40 (mean + std)
# ==========================
for i in range(1, 41):
    columns.append(f"mfcc_delta_{i}_mean")

for i in range(1, 41):
    columns.append(f"mfcc_delta_{i}_std")

# ==========================
# 5. Temporal / Energy
# ==========================
columns += [
    "rms_mean","rms_std",
    "onset_mean","onset_std",
    "tempo",
    "autocorr_lag"
]

# ==========================
# SIMPAN CSV
# ==========================
df = pd.DataFrame(data_rows, columns=columns)
df["label"] = labels

OUTPUT_CSV = r"D:\KULIAH\SEMESTER 5\Program Saint Data\Uranus\myfirstbook\Audio_recognition\features_audio.csv"
df.to_csv(OUTPUT_CSV, index=False)

print(f"\n✅ Fitur disimpan ke {OUTPUT_CSV}")


C:\Users\user\AppData\Local\Temp\ipykernel_18896\4033610634.py:66: FutureWarning: librosa.beat.tempo
	This function was moved to 'librosa.feature.rhythm.tempo' in librosa version 0.10.0.
	This alias will be removed in librosa version 1.0.
  tempo = librosa.beat.tempo(onset_envelope=onset_env, sr=sr)


KeyboardInterrupt: 

### e. Analisa dan pemilihan Fitur terbaik

In [5]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import numpy as np

def feature_importance_analysis(
        X_train, y_train, 
        X_val, y_val, 
        cumulative_threshold=0.95
    ):
    """
    Analisis feature importance menggunakan Random Forest.
    cumulative_threshold: batas cumulative importance (default: 95%)
    """

    print("Training Random Forest untuk analisis feature importance...")
    
    rf_temp = RandomForestClassifier(
        n_estimators=300,
        random_state=42,
        n_jobs=-1
    )
    rf_temp.fit(X_train, y_train)

    # ● Ranking feature importance
    importances = pd.Series(
        rf_temp.feature_importances_, 
        index=X_train.columns
    ).sort_values(ascending=False)

    # ● Hitung cumulative importance secara benar
    cumulative = importances.cumsum()

    # Ambil fitur sampai cumulative >= threshold
    selected_features = cumulative[cumulative <= cumulative_threshold].index.tolist()

    # Jika pas berhenti di tengah, tambahkan 1 fitur berikutnya agar menutup threshold
    if len(selected_features) < len(importances):
        next_feat = importances.index[len(selected_features)]
        selected_features.append(next_feat)

    # Failsafe kalau masih kosong
    if len(selected_features) == 0:
        selected_features = [importances.idxmax()]

    removed_features = [f for f in X_train.columns if f not in selected_features]

    # ● Plot cumulative importance
    plt.figure(figsize=(12, 6))
    plt.plot(np.arange(len(importances)) + 1, cumulative.values, marker='o')
    plt.axhline(y=cumulative_threshold, color='red', linestyle='--')
    plt.xlabel("Jumlah fitur (urutan importance)")
    plt.ylabel("Cumulative importance")
    plt.title("Cumulative Feature Importance (Random Forest)")
    plt.grid(True)
    plt.show()

    # ● Summary
    print("\nHASIL ANALISIS FEATURE IMPORTANCE")
    print("-" * 60)
    print(f"Total fitur awal      : {len(importances)}")
    print(f"Fitur terpilih        : {len(selected_features)}")
    print(f"Fitur terhapus        : {len(removed_features)}")
    print("-" * 60)

    # ● Dataset dengan fitur terpilih
    X_train_sel = X_train[selected_features].copy()
    X_val_sel   = X_val[selected_features].copy()

    return {
        "X_train_selected": X_train_sel,
        "X_val_selected": X_val_sel,
        "selected_features": selected_features,
        "removed_features": removed_features,
        "feature_importances": importances
    }
